### Notation

| symbol | set | code | description |
|:------:|:---:|:----:|:------------|
| $f$    | $\mathbb{R}$ | `myfunc` | the function |
| $\mathbf{X}$ | $\mathbb{R}^n$ | `X` | vector with variables |
| $x_i$ | $\mathbb{R}$ | `x[i]` | the $i$-th variable  |
| $h$ | $\{h \in \mathbb{R} \| h>0 \}$ | `h` | increment parameter (step size, band width) |
| $\frac{\partial f(\mathbf{X})}{\partial x_i}$ | $\mathbb{R}$ | `g[i]` | partial derivative for the $i$-th variable |
| $\nabla f$ | $\mathbb{R}^n$ | `g` | gradient vector |


### Partial Derivatives
Be $\mathbf{X} = (x_i) \in \mathbb{R}^n$ the **variables**, 
$f: \mathbb{R}^n \rightarrow \mathbb{R}$ a (single output) **function**,
$h \in \{h \in \mathbb{R} | h>0 \}$ the **increment** parameter,
$\mathbf{1} = (1_i) \in \{0,1\}^n$ a vector of zeros except the $i$-th entry that equals 1.
The **partial derivative** (or limit) of function $f$ with respect to the $i$-th variable $x_i$ is

$$
\frac{\partial f(\mathbf{X})}{\partial x_i} := \lim_{h \rightarrow 0} \frac{f(\mathbf{X} + h \cdot 1_i) - f(\mathbf{X})}{h} \qquad \forall i
$$

### Gradient Vector
Be $\nabla f \in \mathbb{R}^n$ the **gradient** of $f$ at $x$. 
It is the vector all partial derivatives.

$$
\nabla f := \left(\frac{\partial f(\mathbf{X})}{\partial x_1}, ..., \frac{\partial f(\mathbf{X})}{\partial x_n} \right)
$$

### Finite Difference Approximation
The finite difference approximation method uses a finite small value for the increment parameter, e.g. $h=10^{-8}$, instead of the theoretical *infinitesimal* $\lim_{h \rightarrow 0}$. 
In other words, a finite small value $h$ instead of an infinite small $h$.
On a computer, $h$ depends on the numeric precision to avoid cancellation.

The **forward difference approximation** (Euler method, or secant method) uses the quotient mentioned before

$$
\frac{\partial f(\mathbf{X})}{\partial x_i} 
\approx g^f_i 
= \frac{f(\mathbf{X} + h \cdot 1_i) - f(\mathbf{X})}{h}
$$

It is important to note that 

* the forward approximation is always $g^f_i < \frac{\partial f(\mathbf{X})}{\partial x_i}$ (or bigger) at convex curvatures (or concave curvatures). Similar behavior with backward approximation.
* the forward and backward method will never lie at point $\mathbf{X}$

### C Implementation
The `x` array is temporarily manipulated in `finitediff_forward`. 
In each iteration the `x[i]` value is backed up as `tmp` before adding the bandwith `h`.
At the end `tmp` is reassigned to `x[i]`.
The number of function evaluations is `n+1`.

In [1]:
#include <stdio.h>  

/**************************************************
 ************* finitediff_forward.h ***************
 **************************************************/
void finitediff_forward(double (*f)(double*, int), 
                        double *x, 
                        int n, 
                        double h, 
                        double *g){
    double tmp;
    double f0 = f(x,n);
    for (int i=0; i<n; i++){
        tmp = x[i]; //store temporarly
        x[i] += h;  //add bandwidth param
        g[i] = (f(x,n) - f0) / h;
        x[i] = tmp; //reassign orginal value
    }
} 

/**************************************************
 **************** EXAMPLE SCRIPT ******************
 **************************************************/
double myfunc(double *x, int n){
    double f = 0.0;
    for(int i=0; i<n; i++){
        f += x[i] * x[i];
    }
    f /= (double)n;
    return f;
}

int main(){
    double (*evalfunc)(double*, int) = &myfunc;
    double x[]={-.6, -.5, -.4, -.3, -.2, -.1, 
                0.0, .1 , .2, .3, .4, .5, .6};
    int n = 13;
    double g[n];
    double h = 0.001;
    
    finitediff_forward(evalfunc, x, n, h, g);
    
    for(int i=0; i<n; i++){
        printf("x: %7.4f | g(x): %7.4f \n", x[i], g[i] );
    }
    
    return 0;
}


x: -0.6000 | g(x): -0.0922 
x: -0.5000 | g(x): -0.0768 
x: -0.4000 | g(x): -0.0615 
x: -0.3000 | g(x): -0.0461 
x: -0.2000 | g(x): -0.0307 
x: -0.1000 | g(x): -0.0153 
x:  0.0000 | g(x):  0.0001 
x:  0.1000 | g(x):  0.0155 
x:  0.2000 | g(x):  0.0308 
x:  0.3000 | g(x):  0.0462 
x:  0.4000 | g(x):  0.0616 
x:  0.5000 | g(x):  0.0770 
x:  0.6000 | g(x):  0.0924 
